In [16]:
from analysis import *
import matplotlib.pyplot as plt
import pickle
import os
import numpy as np
from bookkeeper import *

In [18]:
SIM_NAME = "selfish_mine_test"
SIM_REPS = 1
SIM_ITERS = 100000

print(SIM_NAME)
for REP in range(SIM_REPS): 
    print(REP)
    location = f'../dumps/{SIM_NAME}_{REP}'
    nodes = []
    for filename in os.listdir(os.fsencode(location)):
        filename = filename.decode("utf-8")
        with open(f'{location}/{filename}', 'rb') as f:
            if filename ==  'bookkeeper':
                bookkeeper = pickle.load(f)
            else:
                nodes.append(pickle.load(f))

    an = Analysis(bookkeeper, nodes)
            
    print(f'Loaded {len(nodes)} nodes.')

    blocks = an.get_all_blocks()
    print(f'Total blocks:\t{len(blocks)}')

    stale_rates = [an.stale_block_rate(node) for node in nodes]
    print(f'Stale rate:\t{round(np.mean(stale_rates), 5)}')

    block_ints = [an.avg_block_interval(node) for node in nodes]
    print(f'Block interval:\t{np.mean(block_ints)} iters')

    # delays_50 = [an.block_percentile_delay(block, 0.5) for id, block in blocks.items()]
    # delays_90 = [an.block_percentile_delay(block, 0.9) for id, block in blocks.items()]
    # print(f'50% delay:\t{round(np.mean(delays_50), 2)} iters')
    # print(f'90% delay:\t{round(np.mean(delays_90), 2)} iters')

    # tps = an.transactions_per_second(blocks.values(), SIM_ITERS / 10)
    # print(f'Tx per second:\t{tps}')

selfish_mine_test
0
Loaded 20 nodes.
Total blocks:	27
Stale rate:	0.0
Block interval:	42545.44416666667 iters


In [19]:
honest = [node for node in nodes if 'HNST' in node.name]
selfish = [node for node in nodes if 'SFSH' in node.name]
print(len(honest))
print(len(selfish))

12
8


In [20]:
print(honest[0].print_blockchain())

2021-08-10 10:21:14.647 | WARNING  | sim.base_models:print_blockchain:186 - HNST_9
2021-08-10 10:21:14.649 | WARNING  | sim.base_models:print_blockchain:187 - 	BLOCKCHAIN:
2021-08-10 10:21:14.649 | WARNING  | sim.base_models:print_blockchain:189 - 		BLOCK (id:419c4174-9a85-484d-b2a7-e3ed23ab8fe7, prev: None)
2021-08-10 10:21:14.650 | WARNING  | sim.base_models:print_blockchain:189 - 		BLOCK (id:1df8fcc8-5de7-4076-a8a3-10d06539d372, prev: 419c4174-9a85-484d-b2a7-e3ed23ab8fe7)
2021-08-10 10:21:14.651 | WARNING  | sim.base_models:print_blockchain:189 - 		BLOCK (id:2717df8b-914e-426e-b44e-9714fd45d1d0, prev: 1df8fcc8-5de7-4076-a8a3-10d06539d372)
2021-08-10 10:21:14.652 | WARNING  | sim.base_models:print_blockchain:189 - 		BLOCK (id:78a36b62-bd30-42b0-b1c9-1be1e624ef5c, prev: 2717df8b-914e-426e-b44e-9714fd45d1d0)
2021-08-10 10:21:14.654 | WARNING  | sim.base_models:print_blockchain:189 - 		BLOCK (id:8b28e384-1de7-4ea8-a545-c95e74cce4c0, prev: 78a36b62-bd30-42b0-b1c9-1be1e624ef5c)
2021-08-10

None


In [22]:
print(selfish[0].print_blockchain())

2021-08-10 10:21:21.209 | WARNING  | sim.base_models:print_blockchain:186 - SFSH_0
2021-08-10 10:21:21.210 | WARNING  | sim.base_models:print_blockchain:187 - 	BLOCKCHAIN:
2021-08-10 10:21:21.211 | WARNING  | sim.base_models:print_blockchain:189 - 		BLOCK (id:419c4174-9a85-484d-b2a7-e3ed23ab8fe7, prev: None)
2021-08-10 10:21:21.211 | WARNING  | sim.base_models:print_blockchain:190 - 	HEADS:
2021-08-10 10:21:21.212 | WARNING  | sim.base_models:print_blockchain:195 - 		BLOCK (id:419c4174-9a85-484d-b2a7-e3ed23ab8fe7, prev: None)


None


In [23]:
main_chain = an.get_longest_chain(an.get_all_blocks())

In [24]:
for block in main_chain:
    print(block.miner)

HNST_9
HNST_9
HNST_9
HNST_9
